# Targeted Carlini-Wagner (CW) Attack

This notebook demonstrates the targeted Carlini-Wagner attack on Whisper.

**Goal:** Modify an audio sample so Whisper outputs a *specific target phrase* instead of the original content.

**Method:**
1.  Initialize perturbation (random noise).
2.  Optimize perturbation to minimize $L_2$ distance from original audio.
3.  Optimize perturbation to *maximize* the log-probability of the target tokens.
4.  Apply constraints.

In [ ]:
import torch
import numpy as np
import soundfile as sf
import matplotlib.pyplot as plt
from tqdm import tqdm

from src.data.audio_loader import AudioLoader
from src.attacks.cw import CWAuditoryAttack

# Setup Device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

In [ ]:
# Load Model
import whisper

# Using a small model for faster experimentation
model = whisper.load_model("base", device=device)
print("Model loaded.")

In [ ]:
# Load a sample utterance
loader = AudioLoader()
audio_path, text = loader.get_sample()

print(f"Original Audio: {text}")
print(f"Sample path: {audio_path}")

In [ ]:
# Define Target (The phrase we want Whisper to output)
target_phrase = "Hello world"
print(f"Target Phrase: {target_phrase}")

# Run CW Attack
cw_attack = CWAuditoryAttack(
    whisper_model=model,
    device=device,
    learning_rate=0.01,
    c=1.0,
    steps=50
)

# Load raw audio to float32 [-1, 1]
audio_data, _ = sf.read(audio_path)
if audio_data.ndim > 1:
    audio_data = audio_data[:, 0]
clean_audio = torch.tensor(audio_data, dtype=torch.float32)

adv_audio = cw_attack.attack(clean_audio, target_phrase)

# Save results
sf.write('targeted_attack_clean.wav', clean_audio.numpy(), samplerate=16000)
sf.write('targeted_attack_adv.wav', adv_audio.numpy(), samplerate=16000)

print("Results saved.")

In [ ]:
# Evaluate Results
print("\n--- Evaluating Results ---\n")

# Clean
result_clean = model.transcribe('targeted_attack_clean.wav')
print(f"Clean Prediction: {result_clean['text']}")

# Adversarial
result_adv = model.transcribe('targeted_attack_adv.wav')
print(f"Adversarial Prediction: {result_adv['text']}")

# Check SNR (Optional)
# Calculate SNR between clean and adv

### Observations
- Targeted attacks are significantly harder than untargeted attacks (which just try to break the model).
- The perturbation might be larger to successfully override the original semantic information.
- If `result_adv['text']` contains the target phrase, the attack is successful.